# 2. Feature Engineering

In [ ]:
import polars as pl
import matplotlib.pyplot as plot 

from paths import PROCESSED_CSV_FILE, ENGINEERED_CSV_FILE

In [ ]:
df = pl.read_csv(PROCESSED_CSV_FILE)
df.shape

## 1. Explore Data

In [ ]:
df.head()

In [ ]:
df.describe()